<a href="https://colab.research.google.com/github/feijaum/aluraproj/blob/main/ProjetoAlura.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Necessario a API Key do Google AI Studio

In [41]:

!pip install fpdf==1.7.2 -q


import ipywidgets as widgets
from IPython.display import display, Markdown, FileLink
from fpdf import FPDF
import google.generativeai as genai
import google.colab.userdata as userdata
import os

# Configurando a API Key do Google AI Studio
API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=API_KEY)
GEMINI_MODEL = 'gemini-2.5-flash-preview-04-17'

# Funções dos Agentes (mantidas do código anterior)
class AgentePesquisador:
    def processar(self, tema):
        print(f"\nChamando Agente Pesquisador para o tema: {tema}")
        try:
            model = genai.GenerativeModel(GEMINI_MODEL)
            prompt = f"Gere um resumo em tópicos, conciso e dinâmico, sobre o seguinte tema: {tema}. Adapte a linguagem para diferentes níveis de conhecimento."
            response = model.generate_content(prompt)
            return response.text
        except Exception as e:
            return f"Ocorreu um erro ao processar a pesquisa: {e}"

class AgenteQuestionador:
    def processar(self, tema):
        print(f"\nChamando Agente Questionador para o tema: {tema}")
        try:
            model = genai.GenerativeModel(GEMINI_MODEL)
            prompt = f"""Gere um questionário de múltipla escolha com 10 questões específicas sobre "{tema}".
            Inclua o gabarito ao final do questionário.
            Caso o assunto seja muito amplo, foque em um tópico específico dentro dele e indique o tema do questionário no título.
            Adicione uma mensagem ao usuário incentivando a especificidade na busca para questionários mais direcionados.
            Formate o questionário de forma clara, com as questões numeradas e as opções de resposta (a, b, c, d, e)."""
            response = model.generate_content(prompt)
            return response.text
        except Exception as e:
            return f"Ocorreu um erro ao processar o questionário: {e}"

class AgentePlanejador:
    def processar(self, tema):
        print(f"\nChamando Agente Planejador para o tema: {tema}")
        try:
            model = genai.GenerativeModel(GEMINI_MODEL)
            prompt = f"""Elabore um cronograma de estudos de 30 dias para aprender sobre "{tema}" do zero.
            Considere uma disponibilidade de 2 horas de estudo por dia.
            O cronograma deve indicar os temas principais, a ordem de estudo e a alocação de dias para cada tópico."""
            response = model.generate_content(prompt)
            return response.text
        except Exception as e:
            return f"Ocorreu um erro ao processar o cronograma: {e}"

class AgenteProfessor:
    def compilar_info(self, pesquisa, questionario, cronograma):
        print("\nChamando Agente Professor para compilar informações.")
        documento = f"""
# Documento de Estudo Completo

## Pesquisa

{pesquisa}

---

## Quiz

{questionario}

---

## Planejamento de Estudos

{cronograma}

---

*Este documento foi gerado por um assistente de estudos com múltiplos agentes utilizando o Google AI Studio.*
*Em um ambiente real, este conteúdo poderia ser exportado para o Google Docs ou outros formatos.*
        """
        return documento

class AgenteFinal:
    def apresentar_resultado(self, resultado):
        return resultado if resultado is not None else ""

def iniciar_processo(opcao, tema):
    resultado_agente = None

    if opcao == 1:
        agente_pesquisador = AgentePesquisador()
        resultado_agente = agente_pesquisador.processar(tema)

    elif opcao == 2:
        agente_questionador = AgenteQuestionador()
        resultado_agente = agente_questionador.processar(tema)

    elif opcao == 3:
        agente_planejador = AgentePlanejador()
        resultado_agente = agente_planejador.processar(tema)

    elif opcao == 4:
        agente_pesquisador = AgentePesquisador()
        pesquisa = agente_pesquisador.processar(tema)

        agente_questionador = AgenteQuestionador()
        questionario = agente_questionador.processar(tema)

        agente_planejador = AgentePlanejador()
        cronograma = agente_planejador.processar(tema)

        agente_professor = AgenteProfessor()
        resultado_agente = agente_professor.compilar_info(pesquisa, questionario, cronograma)

    if resultado_agente is not None:
        agente_final = AgenteFinal()
        return agente_final.apresentar_resultado(resultado_agente)
    else:
        return "Nenhuma opção válida foi selecionada."

# Frontend (com a adição do botão de download)
label_instrucao = widgets.Label(value="Escolha uma das opções:")
botao_pesquisa = widgets.Button(description="1 - Pesquisa", tooltip="Gera um resumo em tópicos sobre o tema.")
botao_questionario = widgets.Button(description="2 - Questionário", tooltip="Cria um questionário com gabarito sobre o tema.")
botao_cronograma = widgets.Button(description="3 - Cronograma", tooltip="Elabora um cronograma de estudos para o tema.")
botao_completao = widgets.Button(description="4 - Completão", tooltip="Gera pesquisa, questionário e cronograma completos.")
campo_tema = widgets.Text(description="Tema de estudo:")
botao_enviar = widgets.Button(description="Gerar!")
area_resultado = widgets.Output()
botao_baixar_pdf = widgets.Button(description="Baixar como PDF", disabled=True)
output_download = widgets.Output()

# Estilos dos botões (mantidos)
botao_ativo_style = widgets.ButtonStyle(button_color='lightblue')
botao_padrao_style = widgets.ButtonStyle()
botao_ativo = None
opcao_selecionada = None

def resetar_estilo_botoes():
    botao_pesquisa.style = botao_padrao_style
    botao_questionario.style = botao_padrao_style
    botao_cronograma.style = botao_padrao_style
    botao_completao.style = botao_padrao_style

def on_botao_pesquisa_clicked(b):
    global opcao_selecionada, botao_ativo
    opcao_selecionada = 1
    resetar_estilo_botoes()
    botao_pesquisa.style = botao_ativo_style
    botao_ativo = botao_pesquisa

def on_botao_questionario_clicked(b):
    global opcao_selecionada, botao_ativo
    opcao_selecionada = 2
    resetar_estilo_botoes()
    botao_questionario.style = botao_ativo_style
    botao_ativo = botao_questionario

def on_botao_cronograma_clicked(b):
    global opcao_selecionada, botao_ativo
    opcao_selecionada = 3
    resetar_estilo_botoes()
    botao_cronograma.style = botao_ativo_style
    botao_ativo = botao_cronograma

def on_botao_completao_clicked(b):
    global opcao_selecionada, botao_ativo
    opcao_selecionada = 4
    resetar_estilo_botoes()
    botao_completao.style = botao_ativo_style
    botao_ativo = botao_completao

resultado_texto = ""

def gerar_pdf(texto, nome_arquivo="resultado_estudo.pdf"):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)
    # Permite quebras de linha automáticas
    pdf.set_fill_color(200, 220, 255)
    pdf.set_margins(15, 10, 15)
    pdf.set_auto_page_break(auto=True, margin=15)

    if texto:
        for line in texto.split('\n'):
            pdf.multi_cell(0, 5, line)
    else:
        pdf.cell(0, 10, "Nenhum resultado para exportar.", ln=True)

    pdf_bytes = pdf.output(dest='S').encode('latin-1')

    with open(nome_arquivo, 'wb') as f:
        f.write(pdf_bytes)
    return nome_arquivo

def on_botao_enviar_clicked(b):
    global opcao_selecionada, resultado_texto
    with area_resultado:
        area_resultado.clear_output()
        resultado_texto = ""
        botao_baixar_pdf.disabled = True
        output_download.clear_output()
        if opcao_selecionada is not None and campo_tema.value:
            resultado = iniciar_processo(opcao_selecionada, campo_tema.value)
            resultado_texto = resultado
            display(Markdown(resultado))
            botao_baixar_pdf.disabled = False
        elif not campo_tema.value:
            display(widgets.Label(value="Por favor, digite um tema de estudo."))
        else:
            display(widgets.Label(value="Por favor, selecione uma opção."))

def on_botao_baixar_pdf_clicked(b):
    global resultado_texto
    if resultado_texto:
        nome_pdf = gerar_pdf(resultado_texto)
        with output_download:
            output_download.clear_output()
            file_link = FileLink(nome_pdf, result_html_prefix="Clique aqui para baixar o arquivo: ")
            display(file_link)
    else:
        with output_download:
            output_download.clear_output()
            display(widgets.Label(value="Nenhum resultado gerado para baixar."))

botao_pesquisa.on_click(on_botao_pesquisa_clicked)
botao_questionario.on_click(on_botao_questionario_clicked)
botao_cronograma.on_click(on_botao_cronograma_clicked)
botao_completao.on_click(on_botao_completao_clicked)
botao_enviar.on_click(on_botao_enviar_clicked)
botao_baixar_pdf.on_click(on_botao_baixar_pdf_clicked)

rodape_info = widgets.HTML(value='<div style="position: fixed; bottom: 10px; left: 10px; font-size: small;">Assistente de Estudos Versão ALPHA</div><div style="position: fixed; bottom: 10px; right: 10px; font-size: small; text-align: right;">Instagram: <a href="https://www.instagram.com/prof.jvictor/" target="_blank">@prof.jvictor</a><br>LinkedIn: <a href="https://www.linkedin.com/in/jvictorLL/" target="_blank">jvictorLL</a></div>')
logo_alura = widgets.HTML(value='<div style="position: absolute; top: 0px; right: 10px; width: auto; overflow: hidden;"><img src="https://cursos.alura.com.br/assets/images/immersion/imersao_ia_google_logo.png" height="50"></div>')

interface_principal = widgets.VBox([
    widgets.HBox([label_instrucao, logo_alura], layout=widgets.Layout(justify_content='space-between', min_height='60px')),
    widgets.HBox([botao_pesquisa, botao_questionario, botao_cronograma, botao_completao]),
    campo_tema,
    botao_enviar,
    area_resultado,
    botao_baixar_pdf,
    output_download,
], layout=widgets.Layout(margin='0 0 80px 0'))

display(interface_principal)
display(rodape_info)

  Preparing metadata (setup.py) ... done


HTML(value='<div style="position: fixed; bottom: 10px; left: 10px; font-size: small;">Assistente de Estudos Ve…